# 1. Import libraries

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:

!pip install konlpy
!pip install gensim

In [ ]:
#colab에서 실행 중이라면...
!git clone https://github.com/hukim1112/comment_classifier.git
import os
os.chdir('/content/comment_classifier')

In [1]:
import tensorflow as tf
from konlpy.tag import Twitter
from collections import Counter
import pandas as pd
import numpy as np
import gensim
from matplotlib import pyplot as plt

keras = tf.keras
t = Twitter()

/home/dan/.virtualenvs/p3tf/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
/home/dan/.virtualenvs/p3tf/lib/python3.6/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


# 2. fit tokenizer to our datasets

In [11]:
from vectorizer import BaseVectorizer
tokenizer = BaseVectorizer(t.morphs)

In [12]:
df = pd.read_csv('train_intent.csv')

In [13]:
df[500:520]

,question,intent
500,여자 영어 말해줄래요,번역
501,고장 영어 뭔데요,번역
502,방문 영어 뭔지 알아요,번역
503,코끼리 영어 뭔지 알려줄래요,번역
504,파인애플 영어 번역요,번역
505,배 뭔지 영어 번역해줘요,번역
506,전기 번역해주세요,번역
507,물고기 영어 번역해줄래요,번역
508,이 문장 영어 말해줘요,번역
509,이 단어 영어 말해주세요,번역


In [14]:
tokenizer.fit(df['question'].values)

scanning was done                                        
1405 terms are recognized


In [15]:
tokenizer.vocabulary_

{'_PAD_': 0,
 '_STA_': 1,
 '_EOS_': 2,
 '_UNK_': 3,
 '오늘': 4,
 '알려줘': 5,
 '명언': 6,
 '이': 7,
 '누구': 8,
 '알려줘요': 9,
 '날씨': 10,
 '주': 11,
 '지금': 12,
 '해줘': 13,
 '시간': 14,
 '뭐': 15,
 '노래': 16,
 '추천': 17,
 '상태': 18,
 '들려줘': 19,
 '몇': 20,
 '말': 21,
 '공기': 22,
 '이슈': 23,
 '요': 24,
 '무슨': 25,
 '좀': 26,
 '이번': 27,
 '맛집': 28,
 '요즘': 29,
 '나': 30,
 '알려줄래': 31,
 '주변': 32,
 '내일': 33,
 '해줘요': 34,
 '알려주라': 35,
 '지': 36,
 '며칠': 37,
 '알려줄래요': 38,
 '의': 39,
 '좋은': 40,
 '뉴스': 41,
 '사건': 42,
 '현재': 43,
 '이제': 44,
 '곧': 45,
 '번역': 46,
 '날짜': 47,
 '에요': 48,
 '에게': 49,
 '되는': 50,
 '인가요': 51,
 '뭔': 52,
 '거': 53,
 '핫': 54,
 '미세먼지': 55,
 '하나': 56,
 '시야': 57,
 '니': 58,
 '가장': 59,
 '화제': 60,
 '어디': 61,
 '중국어': 62,
 '있나요': 63,
 '이야': 64,
 '요일': 65,
 '인지': 66,
 '힘': 67,
 '들려줘요': 68,
 '일': 69,
 '에': 70,
 '마스크': 71,
 '어떻게': 72,
 '우산': 73,
 '해주라': 74,
 '가': 75,
 '해주세요': 76,
 '멋진': 77,
 '결과': 78,
 '바람': 79,
 '프랑스어': 80,
 '다음': 81,
 '있니': 82,
 '전': 83,
 '누군지': 84,
 '알려주세요': 85,
 '라': 86,
 '영어': 87,
 '음악': 88,
 '시': 89,


# 3. data preprocessing

In [16]:
label_to_id = {t:i for i,t in enumerate(df.intent.unique())}
id_to_label = {i:t for i,t in enumerate(df.intent.unique())}

In [17]:
print(label_to_id)

{'음악': 0, '번역': 1, '뉴스': 2, '명언': 3, '달력': 4, '먼지': 5, '인물': 6, '시간': 7, '이슈': 8, '날씨': 9, '맛집': 10}


In [18]:
# df.intent = df.intent.map(lambda x : label_index[x])
# print(df.head(10))

In [19]:
MAX_LENGTH = 40
def tokenize_and_filter(sentences, labels):
  inputs, outputs = [], []
  
  for sentence, label in zip(sentences, labels):
    # tokenize sentence
    tokenized_sentence = tokenizer.encode_a_doc_to_list(sentence)
    # check tokenized sentence max length
    if len(tokenized_sentence) <= MAX_LENGTH:
      inputs.append(tokenized_sentence)
      outputs.append(label_to_id[label])
  
  # pad tokenized sentences
  padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      inputs, maxlen=MAX_LENGTH, padding='post', 
      value = tokenizer.vocabulary_['_PAD_']) # value = 0
  
  return padded_inputs, outputs

In [20]:
inputs, outputs = tokenize_and_filter(df.question, df.intent)

In [21]:
print('encoded input : ', inputs[0], 'label : ', outputs[0], 'original input sentence : ', tokenizer.decode_from_list(inputs[0]))

encoded input :  [185  73  19   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0] label :  0 original input sentence :  ['에픽하이', '우산', '들려줘', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_']


In [28]:
BATCH_SIZE = 16
BUFFER_SIZE = 7836

# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [29]:
for x, y in dataset.take(1):
    print(x, y)
    print('-----------------------------------------------')
    print(x.shape, y.shape)

tf.Tensor(
[[   4   47    9    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [ 152   28   61  101    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [  33   22   18  144  178    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [ 188   11 1027   78   38    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [  54  115   53   31    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 

# 4. model design

In [30]:
print(len(label_to_id.values()))

11


In [31]:
def get_model():
    return tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.n_vocabs, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(label_to_id.values()), activation='softmax')
])

In [32]:
model = get_model()

In [33]:
LEARNING_RATE = 0.0001

In [34]:
model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [35]:
model.fit(dataset, epochs=10)

W0903 22:13:20.885957 140222202435392 deprecation.py:323] From /home/dan/.virtualenvs/p3tf/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
225/225 [==============================] - 4s 16ms/step - loss: 2.3831 - sparse_categorical_accuracy: 0.1665
Epoch 2/10
225/225 [==============================] - 1s 6ms/step - loss: 2.2805 - sparse_categorical_accuracy: 0.3311
Epoch 3/10
225/225 [==============================] - 1s 6ms/step - loss: 1.6393 - sparse_categorical_accuracy: 0.5948
Epoch 4/10
225/225 [==============================] - 1s 6ms/step - loss: 1.0978 - sparse_categorical_accuracy: 0.8065
Epoch 5/10
225/225 [==============================] - 1s 6ms/step - loss: 0.7358 - sparse_categorical_accuracy: 0.8978
Epoch 6/10
225/225 [==============================] - 1s 6ms/step - loss: 0.5000 - sparse_categorical_accuracy: 0.9499
Epoch 7/10
225/225 [==============================] - 1s 6ms/step - loss: 0.3525 - sparse_categorical_accuracy: 0.9671
Epoch 8/10
225/225 [==============================] - 1s 5ms/step - loss: 0.2569 - sparse_categorical_accuracy: 0.9766
Epoch 9/10
225/225 [===========================

In [36]:
def question_processing(sentences):
    inputs = []
    for sentence in sentences:
        # tokenize sentence
        tokenized_sentence = tokenizer.encode_a_doc_to_list(sentence)
        # check tokenized sentence max length
        if len(tokenized_sentence) <= MAX_LENGTH:
            inputs.append(tokenized_sentence)
        else:
            print('입력이 너무 길어요.')
    # pad tokenized sentences
    padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    inputs, maxlen=MAX_LENGTH, padding='post', 
    value = tokenizer.vocabulary_['_PAD_']) # value = 0
    return padded_inputs

In [37]:
input_sentence = question_processing(['서울 날씨 어때?', 
                                      '나는 전주 날씨 궁금함',
                                      '안중근 의사는 누구야?',
                                      '이순신 장군은 어떤 분이니?',
                                      '명동 맛있는 음식점 있니?'
                                     ])

In [38]:
model.predict(input_sentence)

array([[1.8714919e-03, 8.0991877e-06, 1.3600898e-02, 1.0793593e-04,
        1.0797166e-03, 8.9184325e-03, 5.0976087e-05, 5.6394748e-04,
        1.0565180e-04, 9.4367051e-01, 3.0022357e-02],
       [2.2109991e-03, 1.9500159e-05, 3.8367338e-02, 1.6750961e-03,
        1.8929126e-04, 1.3565199e-03, 1.9754931e-05, 5.5878405e-05,
        2.6051935e-06, 9.1036278e-01, 4.5740198e-02],
       [8.9842110e-04, 4.2620990e-02, 6.1833546e-02, 9.9253962e-03,
        2.0856133e-02, 1.5820228e-04, 8.6182660e-01, 3.0788023e-04,
        8.3963769e-06, 9.0734065e-05, 1.4738131e-03],
       [1.4561039e-01, 2.0039240e-02, 1.1131429e-02, 1.9469393e-04,
        8.6124696e-02, 1.9042899e-01, 6.5536886e-02, 3.1211960e-01,
        1.3397595e-01, 4.5647430e-03, 3.0273316e-02],
       [4.1288920e-02, 3.6198052e-03, 4.2151101e-02, 2.5887890e-03,
        7.3821942e-04, 9.3847082e-04, 7.8636799e-03, 1.6212465e-02,
        3.7641199e-05, 1.6968068e-02, 8.6759287e-01]], dtype=float32)

In [39]:
prediction = np.argmax(model.predict(input_sentence), axis=1)
print(prediction)

[ 9  9  6  7 10]


In [40]:
for p in prediction:
    print(id_to_label[p])

날씨
날씨
인물
시간
맛집


In [27]:
del model

# 데이터 추가해보기

In [41]:
names = ['안중근', '이순신', '세종대왕', '김광석', '아이유', '에미넴', '이건희', '고아라', '유재석', '한석희', '최민성']
def question_generator(names):
    question = []
    for name in names:
        s1 = name+'는 어떤 분이야?'
        s2 = name+'은 어떤 사람이니?'
        s3 = name+'이란 사람에 대해 궁금해'
        question = question+[s1, s2, s3]
    return question
question = question_generator(names)

In [42]:
question

['안중근는 어떤 분이야?',
 '안중근은 어떤 사람이니?',
 '안중근이란 사람에 대해 궁금해',
 '이순신는 어떤 분이야?',
 '이순신은 어떤 사람이니?',
 '이순신이란 사람에 대해 궁금해',
 '세종대왕는 어떤 분이야?',
 '세종대왕은 어떤 사람이니?',
 '세종대왕이란 사람에 대해 궁금해',
 '김광석는 어떤 분이야?',
 '김광석은 어떤 사람이니?',
 '김광석이란 사람에 대해 궁금해',
 '아이유는 어떤 분이야?',
 '아이유은 어떤 사람이니?',
 '아이유이란 사람에 대해 궁금해',
 '에미넴는 어떤 분이야?',
 '에미넴은 어떤 사람이니?',
 '에미넴이란 사람에 대해 궁금해',
 '이건희는 어떤 분이야?',
 '이건희은 어떤 사람이니?',
 '이건희이란 사람에 대해 궁금해',
 '고아라는 어떤 분이야?',
 '고아라은 어떤 사람이니?',
 '고아라이란 사람에 대해 궁금해',
 '유재석는 어떤 분이야?',
 '유재석은 어떤 사람이니?',
 '유재석이란 사람에 대해 궁금해',
 '한석희는 어떤 분이야?',
 '한석희은 어떤 사람이니?',
 '한석희이란 사람에 대해 궁금해',
 '최민성는 어떤 분이야?',
 '최민성은 어떤 사람이니?',
 '최민성이란 사람에 대해 궁금해']

In [43]:
new_data = {'question' : question, 'intent' : ['인물']*len(question)}
add_df = pd.DataFrame(new_data, columns=('question', 'intent'))

In [44]:
add_df.head(5)

,question,intent
0,안중근는 어떤 분이야?,인물
1,안중근은 어떤 사람이니?,인물
2,안중근이란 사람에 대해 궁금해,인물
3,이순신는 어떤 분이야?,인물
4,이순신은 어떤 사람이니?,인물


In [45]:
print(len(df), len(add_df))

3591 33


In [46]:
new_df = pd.concat([df, add_df])
print(len(new_df))

3624


In [47]:
tokenizer.fit(new_df['question'].values)

scanning was done                                        
1413 terms are recognized


In [48]:
new_inputs, new_outputs = tokenize_and_filter(new_df.question, new_df.intent)

BATCH_SIZE = 16
BUFFER_SIZE = 7836

# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((new_inputs, new_outputs))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [49]:
new_model = get_model()
LEARNING_RATE = 0.0001
new_model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
new_model.fit(dataset, epochs=10)

Epoch 1/10
227/227 [==============================] - 3s 12ms/step - loss: 2.3804 - sparse_categorical_accuracy: 0.2039
Epoch 2/10
227/227 [==============================] - 1s 6ms/step - loss: 2.1885 - sparse_categorical_accuracy: 0.3675
Epoch 3/10
227/227 [==============================] - 1s 6ms/step - loss: 1.5188 - sparse_categorical_accuracy: 0.6407
Epoch 4/10
227/227 [==============================] - 1s 6ms/step - loss: 1.0284 - sparse_categorical_accuracy: 0.8132
Epoch 5/10
227/227 [==============================] - 1s 6ms/step - loss: 0.7007 - sparse_categorical_accuracy: 0.9092
Epoch 6/10
227/227 [==============================] - 1s 6ms/step - loss: 0.4857 - sparse_categorical_accuracy: 0.9506
Epoch 7/10
227/227 [==============================] - 1s 6ms/step - loss: 0.3459 - sparse_categorical_accuracy: 0.9655
Epoch 8/10
227/227 [==============================] - 1s 6ms/step - loss: 0.2532 - sparse_categorical_accuracy: 0.9763
Epoch 9/10
227/227 [===========================

In [50]:
input_sentence = question_processing(['서울 날씨 어때?', 
                                      '나는 전주 날씨 궁금함',
                                      '안중근 의사는 누구야?',
                                      '박소희는 어떤 사람인지 궁금해.',
                                      '명동 맛있는 음식점 있니?'
                                     ])

In [51]:
new_model.predict(input_sentence)

array([[1.7098315e-05, 2.3708965e-04, 1.3574773e-02, 1.8590025e-06,
        1.2115539e-03, 7.8989016e-03, 4.0217906e-02, 9.7181126e-03,
        5.9514725e-05, 9.2693919e-01, 1.2393764e-04],
       [3.8869541e-05, 6.8817302e-05, 2.3142852e-02, 7.9726067e-07,
        9.4674528e-04, 1.4378503e-02, 7.5284531e-03, 3.4775245e-03,
        1.5412469e-04, 9.5022339e-01, 3.9826009e-05],
       [2.9938606e-06, 1.7338196e-03, 8.8860317e-05, 8.7949279e-04,
        9.8213041e-03, 4.2381487e-04, 9.8534518e-01, 2.9079005e-05,
        3.3700932e-08, 5.7981297e-04, 1.0955815e-03],
       [1.8741618e-04, 1.9805697e-03, 1.2764053e-03, 1.8994873e-03,
        3.9683390e-01, 1.6610209e-02, 5.7167000e-01, 4.6586578e-05,
        1.0542994e-06, 4.1515138e-03, 5.3428588e-03],
       [3.0631936e-04, 2.7119558e-02, 2.8930921e-03, 1.4638201e-03,
        2.2176368e-02, 1.8066692e-04, 3.7769955e-01, 5.3453497e-03,
        3.0555071e-05, 6.8365422e-04, 5.6210101e-01]], dtype=float32)

In [52]:
prediction = np.argmax(new_model.predict(input_sentence), axis=1)

In [53]:
for p in prediction:
    print(id_to_label[p])

날씨
날씨
인물
인물
맛집
